<a href="https://colab.research.google.com/github/vahidNaghshin/Data_structures_and_algorithms_in_Python/blob/main/Data_structures_and_algorithms_in_Python_Chapter_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
words = 'counter, guranted'
words.split()

['counter,', 'guranted']

In [2]:
freq = {'vahid': 35, 'sahel': 26}
freq

{'vahid': 35, 'sahel': 26}

In [3]:
freq.get('vahid', 3)

35

In [4]:
words[0].isalpha()

True

In [5]:
freq={}

filename = '/content/sample_data/README.md'

for piece in open(filename).read().lower().split():
  # print(piece)
  word = ''.join(c for c in piece if c.isalpha())
  # print(word)
  # print('----')
  if word: # require at least one alphabetic character
    freq[word] = 1 + freq.get(word, 0)
  max_word = ''
  max_count = 0
  for (w, c) in freq.items():
    if c > max_count: 
      max_word = w 
      max_count = c

print( 'The most frequent word is ' , max_word) 
print( 'Its number of occurrences is ' , max_count)



The most frequent word is  is
Its number of occurrences is  4


In [6]:
from collections.abc import MutableMapping

class MapBase(MutableMapping):
  class _Item:
    __slots__ = '_key' , '_value'
    def __init__ (self, k, v): 
      self._key = k
      self._value = v
    
    def __eq__(self, other):
      return self._key == other._key
    
    def __ne__ (self, other): 
      return not (self == other)
    
    def __lt__ (self, other):
      return self._key < other._key

In [7]:
class UnsortedTableMap(MapBase):
  def __init__ (self):
    self._table = []
  
  def __getitem__ (self, k):
    for item in self._table:
      if k == item._key: 
        return item._value
    
    raise KeyError( 'Key Error: '  + repr(k))

  def __setitem__ (self, k, v):
    for item in self._table:
      if k == item._key: 
        item._value = v 
        return
    # did not find match for key
    self._table.append(self._Item(k,v)) 
  
  def __delitem__ (self, k):
    for j in range(len(self._table)):
      if k == self._table[j]._key: 
        self._table.pop(j)
        return
    raise KeyError( 'Key Error: '  + repr(k))
  
  def __len__ (self):
    return len(self._table)

  def __iter__ (self):
    for item in self._table:
      yield item._key 

  def items(self):
    for item in self._table:
      yield (item._key , item._value)

In [8]:
h = UnsortedTableMap()
h['1'] = 2
h['2'] = 3
h['3'] = 5

In [9]:
h.items()

<generator object UnsortedTableMap.items at 0x7f306f1e0740>

In [10]:
'3' in h

True

In [11]:
h.pop('9293', 9)

9

In [12]:
for i in h.items():
  print(i)

('1', 2)
('2', 3)
('3', 5)


In [13]:
def hash_code(s):
  mask = (1 << 32) - 1
  h = 0
  for character in s:
    h = (h << 5 & mask) | (h >> 27)
    h += ord(character)
  
  return h



In [14]:
s = 'u'
hash_code(s)

117

In [15]:
2 >> 4

0

In [16]:
hash(5)

5

In [17]:
hash(5.0)

5

In [18]:
5 == 5.0

True

In [19]:
import random
class HashMapBase(MapBase):
  def __init__ (self, cap=11, p=109345121): 
     self._table = cap * [ None ]
     self._n = 0
     self._prime = p
     self._scale = 1 + random.randrange(p - 1) 
     self._shift = random.randrange(p)
  
  def _hash_function(self, k):
    return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

  def __len__ (self): 
    return self._n
  
  def __getitem__ (self, k):
    j = self._hash_function(k) 
    return self._bucket_getitem(j, k)
  
  def __setitem__ (self, k, v):
    j = self._hash_function(k)
    self._bucket_setitem(j, k, v)
    if self._n > len(self._table) // 2:
      self._resize(2 * len(self. table) - 1)
  
  def __delitem__ (self, k):
    j = self._hash_function(k) 
    self._bucket_delitem(j, k)
    self._n -= 1

  def _resize(self, c):
    old = list(self.items()) 
    self._table = c * [None] 
    self._n = 0
    for (k,v) in old:
      self[k] = v

In [20]:
class ChainHashMap(HashMapBase):
  def _bucket_getitem(self, j, k): 
    bucket = self._table[j]
    if bucket is None:
      raise KeyError( 'Key Error: '  + repr(k)) 
    return bucket[k]
  
  def _bucket_setitem(self, j, k, v): 
    if self._table[j] is None:
      self._table[j] = UnsortedTableMap() 
    oldsize = len(self._table[j])
    self._table[j][k] = v
    if len(self._table[j]) > oldsize:
      self._n += 1
  
  def _bucket_delitem(self, j, k): 
    bucket = self._table[j]
    if bucket is None:
      raise KeyError( 'Key Error: '  + repr(k)) 
    del bucket[k]

  def __iter__ (self):
    for bucket in self._table:
      if bucket is not None: 
        for key in bucket:
          yield key


In [21]:
class ProbeHashMap(HashMapBase):
  _AVAIL = object()

  def _is_available(self, j):
    return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

  def _find_slot(self, j, k):

    firstAvail = None
    while True:
      if self._is_available(j): 
        if firstAvail is None:
          firstAvail = j
        if self._table[j] is None:
          return (False, firstAvail)
      elif k == self._table[j]._key:
        return (True, j)
      
      j = (j + 1) % len(self._table)

  def _bucket_getitem(self, j, k): 
    found, s = self._find_slot(j, k) 
    if not found:
      raise KeyError( 'Key Error: '  + repr(k)) 
    return self._table[s]._value


  def _bucket_setitem(self, j, k, v): 
    found, s = self._find_slot(j, k) 
    if not found:
      self._table[s] = self._Item(k,v)
      self._n += 1 
    else:
      self._table[s]._value = v
  
  def _bucket_delitem(self, j, k): 
    found, s = self._find_slot(j, k) 
    if not found:
      raise KeyError( 'Key Error:  ' + repr(k)) 
    self._table[s] = ProbeHashMap._AVAIL
  
  def __iter__ (self):
    for j in range(len(self._table)):
      if not self._is_available(j): 
        yield self._table[j]._key

In [22]:
H = ProbeHashMap()

In [23]:
H[91] = 14

In [24]:
H[19, 54] = 14

In [25]:
for i in H:
  print(i)

91
(19, 54)


In [26]:
class SortedTableMap(MapBase):
  def _find_index(self, k, low, high):
    if high < low: 
      return high + 1
    else:
      mid = (low + high) // 2
      if k == self._table[mid]._key:
        return mid
      elif k < self._table[mid]._key:
        return self._find_index(k, low, mid - 1) 
      else:
        return self._find_index(k, mid + 1, high)
  
  def __init__ (self):
    self._table = []
  
  def __len__ (self):
    return len(self._table)
  
  def __getitem__ (self, k):
    j = self._find_index(k, 0, len(self._table) - 1)
    if j == len(self._table) or self._table[j]._key != k:
      raise KeyError( 'Key Error: '  + repr(k)) 
    return self._table[j]._value
  
  def __setitem__ (self, k, v):
     j = self._find_index(k, 0, len(self._table) - 1)
     if j < len(self._table) and self._table[j]._key == k:
       self._table[j]._value = v 
     else:
       self._table.insert(j, self._Item(k,v))
  
  def __delitem__ (self, k):
    j = self._find_index(k, 0, len(self._table) - 1)
    if j == len(self._table) or self._table[j]._key != k:
        raise KeyError( 'Key Error: '  + repr(k)) 
    self._table.pop(j)
  
  def __iter__ (self):
    for item in self._table:
      yield item._key
  
  def __reversed__ (self):
    for item in reversed(self._table):
      yield item._key
  
  def find_min(self):
    if len(self._table) > 0:
      return (self._table[0]._key, self._table[0]._value) 
    else:
      return None
  
  def find_max(self):
    if len(self._table) > 0:
      return (self._table[-1]._key, self._table[-1]._value) 
    else:
      return None
  
  def find_ge(self, k):
    j = self._find_index(k, 0, len(self._table) - 1) 
    if j < len(self._table):
      return (self._table[j]._key, self._table[j]._value) 
    else:
      return None
  
  def find_lt(self, k):
    j = self._find_index(k, 0, len(self._table) - 1) 
    if j > 0:
      return (self._table[j-1]._key, self._table[j-1]._value)
    else:
      return None
  
  def find_gt(self, k):
    j = self._find_index(k, 0, len(self._table) - 1) # j s key >= k 
    if j < len(self._table) and self._table[j]._key == k:
      j += 1
    if j < len(self._table):
      return (self._table[j]._key, self._table[j]._value) 
    else:
      return None
  
  def find_range(self, start, stop):
    if start is None:
      j = 0
    else:
      j = self._find_index(start, 0, len(self._table) - 1)
    while j < len(self._table) and (stop is None or self._table[j]._key < stop):
      yield (self._table[j]._key, self._table[j]._value) 
      j += 1


  




      



In [27]:
class CostPerformanceDatabase:
  def init (self):
    self._M = SortedTableMap()
  
  def best(self, c):
    return self._M.find_le(c)
  
  def add(self, c, p):
    other = self._M.find_le(c) 
    if other is not None and other[1] >= p: 
      return
    self._M[c] = p
    other = self._M.find_gt(c)
    while other is not None and other[1] <= p:
      del self._M[other[0]]
      other = self._M.find_gt(c)

In [28]:
from collections.abc import MutableSet

class TestSet(MutableSet):
  def __lt__ (self, other):
    if len(self) >= len(other):
      return False 
    for e in self:
      if e not in other: 
        return False
    return True

  def __or__ (self, other): 
    result = type(self)() 
    for e in self:
      result.add(e) 
    for e in other:
      result.add(e) 
    return result
  

In [29]:
S = TestSet()

TypeError: ignored

In [30]:
class X:
  def __init__(self, c):
    self._c = c
  def instantiate_another(self, v):
    return type(self)(v)

In [31]:
K=X(4)

In [32]:
K._c

4

In [33]:
H = K.instantiate_another(9)
H._c

9

In [34]:
def __ior__ (self, other): 
  for e in other:
    self.add(e)
  return self

In [35]:
class MultiMap:
  _MapType = dict

  def __init__ (self):
    self._map = self._MapType()
  
  def __iter__ (self):
    for k, secondary in self._map.items():
      for v in secondary: 
        yield (k,v)
  
  def add(self, k, v):
    container = self._map.setdefault(k, []) 
    container.append(v)
    self._n += 1
  
  def pop(self, k):
    secondary = self._map[k] 
    v = secondary.pop()
    if len(secondary) == 0:
      del self._map[k] 
    self._n -= 1 
    return (k, v)
  
  def find(self, k):
    secondary = self._map[k] 
    return (k, secondary[0])
  
  def find_all(self, k):
    secondary = self._map.get(k, []) # empty list, by default 
    for v in secondary:
      yield (k,v)

# Reinforcement

In [ ]:
#@title R-10.1
#@markdown It is asseumed that the five basic methods of MutableMapping class is impelmented
def pop(self, k, d=None):
    if k in self:
      return self[k]
    else:
      return d  

In [ ]:
#@title R-10.2
#@markdown It is asseumed that the five basic methods of MutableMapping class is impelmented

def items(self):
    for k in self:
      yield (k, self[k])

In [ ]:
#@title R-10.3
def items(self):
    for item in self._table:
      yield (item._key , item._value)

In [ ]:
#@title R-10.5
class UnsortedTableMap(MapBase):
  def __init__ (self):
    self._table = PositionalList()
  
  def __getitem__ (self, k):
    for p in self._table:
      if k == p.element._Item._key: 
        return p.element._Item._value
    
    raise KeyError( 'Key Error: '  + repr(k))

  def __setitem__ (self, k, v):
    for p in self._table:
      if k == p.element._Item._key: 
        p.element._Item._value = v 
        return
    # did not find match for key
    self._table.add_first(self._Item(k, v)) 
  
  def __delitem__ (self, k):
    for p in self._table:
      if k == p.element._Item._key: 
        self._table.delete(p)
        return
    raise KeyError( 'Key Error: '  + repr(k))

  
  def __len__ (self):
    return len(self._table)

  def __iter__ (self):
    for p in self._table:
      yield p.element._Item._key 

In [ ]:
#@title R-10.7


In [ ]:
#@title R-10.9
import numpy as np

@np.vectorize
def hash_code(x):
  return ((3 * x) + 5 ) % 11

In [ ]:
x = [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]
hash_code(x)

array([ 8,  5,  0,  5,  8,  1,  5,  1, 10,  9,  9])

In [ ]:
#@title R-10.15
import random, math
class HashMapBase(MapBase):
  def __init__ (self, max_load=0.5, cap=11, p=109345121): 
     self._table = cap * [ None ]
     self._n = 0
     self._prime = p
     self._scale = 1 + random.randrange(p - 1) 
     self._shift = random.randrange(p)
     self._max_laod = max_load
  
  def _hash_function(self, k):
    return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

  def __len__ (self): 
    return self._n
  
  def __getitem__ (self, k):
    j = self._hash_function(k) 
    return self._bucket_getitem(j, k)
  
  def __setitem__ (self, k, v):
    j = self._hash_function(k)
    self._bucket_setitem(j, k, v)

    w = math.ceil(1/self.max_load)



    if self._n > len(self._table) // w:
      self._resize(w * len(self. table) - 1)
  
  def __delitem__ (self, k):
    j = self._hash_function(k) 
    self._bucket_delitem(j, k)
    self._n -= 1

  def _resize(self, c):
    old = list(self.items()) 
    self._table = c * [None] 
    self._n = 0
    for (k,v) in old:
      self[k] = v

In [ ]:
#@title R-10.17
class ProbeHashMap(HashMapBase):
  """Hash map implemented with quadratic probing for collision resolution."""
  _AVAIL = object()

  def _is_available(self, j):
    return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

  def _find_slot(self, j, k):

    i = 0

    firstAvail = None
    while True:
      if self._is_available(j): 
        if firstAvail is None:
          firstAvail = j
        if self._table[j] is None:
          return (False, firstAvail)
      elif k == self._table[j]._key:
        return (True, j)
      
      j = (j + i**2) % len(self._table)
      i += 1

  def _bucket_getitem(self, j, k): 
    found, s = self._find_slot(j, k) 
    if not found:
      raise KeyError( 'Key Error: '  + repr(k)) 
    return self._table[s]._value

  def _bucket_setitem(self, j, k, v): 
    found, s = self._find_slot(j, k) 
    if not found:
      self._table[s] = self._Item(k,v)
      self._n += 1 
    else:
      self._table[s]._value = v
  
  def _bucket_delitem(self, j, k): 
    found, s = self._find_slot(j, k) 
    if not found:
      raise KeyError( 'Key Error:  ' + repr(k)) 
    self._table[s] = ProbeHashMap._AVAIL
  
  def __iter__ (self):
    for j in range(len(self._table)):
      if not self._is_available(j): 
        yield self._table[j]._key

In [ ]:
#@title R-10.24
#pseudo-code description of the __delitem__ map operation when using a skip list.
Algorithm __delitem__ (k):
"""delete the position with key k and return the corresponding value"""
Input: Key k
Output: The corresponding value with key k
p = SkipSearch(k)
p = next(p)
while above(p) is None do:
  k = p
  w = next(p)
  q = prev(p)

  delete p
  next(q) = w
  prev(w) = q 

  p = above(k)



SyntaxError: ignored

In [ ]:
#@title R-10.25
from collections import MutableSet
def pop(self):
  if len(self) == 0:
    raise Empty("The set is empty!")
  else:
    for e in self:   #there is no guarantee in order of elements of set
      return e

<ipython-input-43-54f28932addb>:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import MutableSet


In [ ]:
#@title R-10.26
def isdisjoint(self, other):
  for e in self:
    if e in other:
      return False
  return True

# Creativity

In [ ]:
#@title C-10.28
def setdefault(self, k, d):
    for item in self._table:
      if k == item._key:
        return item._value
    self._table.append(self._Item(k, d))
    return d 

In [36]:
#@title C-10.29

# This is the method which should be implemented in ProbeHashMap
def _bucket_setdefault(self, j, k, d):
      found, s = self._find_slot(j, k)
      if not found:
        self._table[s] = self._Item(k, d)
        self._n += 1
        return d
      else:
        return self._table[s]._value



# This is the method which should be implemented in HashMapBase
def setdefault(self, k, d):
    j = self._hash_function(k)
    return self._bucket_setdefault(self, j, k, d)



In [37]:
#@title C-10.30

# This is the method which should be implemented in ProbeHashMap
def _bucket_setdefault(self, j, k, d):
    bucket = self._table[j]
    if self._table[j] is None:
      self._bucket_setitem(j, k, d)
    else:
      self._bucket_getitem(self, j, k)


# This is the method which should be implemented in ChainHashMap
def setdefault(self, k, d):
    j = self._hash_function(k)
    return self._bucket_setdefault(j, k, d)


In [52]:
#@title C-10.31

def is_prime(n):
  for i in range(2,n):
    if (n % i) == 0:
      return False
  return True 

def sieve_algorithm(M):
  table = [True] * (2 * M)
  table[0] = False
  for i in range(2, math.ceil(math.sqrt(2 * M))):
    for k in range(1, math.ceil(2*M/i)):
      if not is_prime(i*k):
        table[i*k] = False

  return table

In [54]:
#@title C-10.32

In [88]:
#@title C-10.33
def _bucket_getitem(self, j, k): 
    bucket = self._table[j]
    if bucket is None:
      raise KeyError( 'Key Error: '  + repr(k))

    if isinstance(bucket, self._Item):
      return bucket._value
    else:
      return bucket[k]

def _bucket_setitem(self, j, k, v): 
  if self._table[j] is None:
    self._table[j] = self._Item(k, v)
    self._n += 1
  elif isinstance(self._table[j], self._Item):
    old_value = self._table[j]
    self._table[j] = UnsortedTableMap()
    self._table[j][old_value._key] = old_value._value
    self._table[j][k] = v
    oldsize = len(self._table[j])
  elif isinstance(self._table[j], UnsortedTableMap):
    self._table[j][k] = v
    oldsize = len(self._table[j])

  if isinstance(self._table[j], UnsortedTableMap) and len(self._table[j]) > oldsize:
    self._n += 1
    

In [89]:
#@title C-10.34

class MapBase(MutableMapping):
  class _Item:
    __slots__ = '_key' , '_value', '_hcode'
    def __init__ (self, k, v): 
      self._key = k
      self._value = v
      self._hcode = None #is set whenever the hash code is determined
    
    def __eq__(self, other):
      return self._key == other._key
    
    def __ne__ (self, other): 
      return not (self == other)
    
    def __lt__ (self, other):
      return self._key < other._key


In [36]:
#@title C-10.35
#we can append an attribute that declares if a given element is collision type or not. If one element is removed, we shift all element one cell to the left to fill the cells.

In [ ]:
#@title C-10.38
ProbeHashMap